<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# script para que funcione selenium en colab

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.j9ClAiQFIC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.9YGsUyBJN6/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.33ovatwgiA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-

# Scraper

## imports

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## funciones

In [3]:
url = 'https://raw.githubusercontent.com/cristianbossolasco/scraper-agencias-viajes/main/busplus_files/paradas.csv'
df_paradas = pd.read_csv(url, delimiter = ';', encoding = 'unicode_escape')
df_paradas.nombre = df_paradas.nombre.apply(lambda x: x.encode('iso-8859-1').decode('utf-8'))
df_paradas.head()

,id,direccion,latitud,longitud,nombre
0,6,,-35137317600,-6.045238e+10,Bragado - Caminera (Buenos Aires - ARG)
1,7,,-35838385600,-6.187472e+10,Pehuajo - ESSO (Buenos Aires - ARG)
2,8,,-35636855700,-6.135356e+10,Carlos Casares - Shell (Buenos Aires - ARG)
3,10,,-34905308000,-5.795551e+10,La Plata - Terminal (Buenos Aires - ARG)
4,11,,-35451965600,-6.088127e+10,9 de Julio - Terminal (Buenos Aires - ARG)


In [4]:
def get_data(driver, origen_parada, nombre_origen_parada, destino_parada, nombre_destino_parada, fecha_ida):
  """
    Esta funcion es la encargada de hacer el scraper de un origen y destino puntual en una fecha dada 

    Args:
      origen_parada (str): id de la parade de origen
      nombre_origen_parada (str): El nombre de la parada de origen que sera el que figure en los registros del output
      destino_parada (str): id de la parada de destino
      nombre_destino_parada (str): El nombre de la parada de destino que sera el que figure en los registros del output
      fecha_ida (str): fecha de partida en el formato {year}/{month}/{day}

    Returns:
      lista de json con los viajes
  """

  url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='
  driver.get(url)

  # get_attribute('innerHTML') para obtener el html

  clean_hora = lambda x: re.search(r'(\d+):(\d+)', str(x)).group() if re.search(r'(\d+):(\d+)', str(x)) else 'None'
  clean_precio = lambda x: re.search(r'(\d+)', str(x)).group() if re.search(r'(\d+)', str(x)) else 'None'


  items = []
  filas  = driver.find_elements(By.CLASS_NAME,"fila_servicio.servicio_fila.idatr")
  # recorro los registros de viajes
  for fila in filas:
    if not "hide " in fila.get_attribute(name="class"):
      id = fila.get_attribute('id')

      empresa_img = fila.find_element(By.TAG_NAME,"td").find_element(By.TAG_NAME, 'img').get_attribute('src')
      salida = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[2]').text.replace('\n', ' ')
      llegada = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[4]').text.replace('\n', ' ')
      escala = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[5]/a').text.replace('\n', ' ')
      duracion = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[6]').text.replace('\n', ' ')
      duracion = clean_hora(duracion)
      categoria = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[7]/ul/li[1]').text
      butacas_libres = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[8]/ul/li[1]').text
      precio = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[9]/ul/li[1]/span').text
      precio = clean_precio(precio)

      item_json = {
          "origen_parada": origen_parada,
          "nombre_origen_parada": nombre_origen_parada,
          "destino_parada": destino_parada,
          "nombre_destino_parada": nombre_destino_parada,
          "fecha_ida": fecha_ida,
          "empresa_img": empresa_img,
          "salida": salida,
          "llegada": llegada,
          "escala": escala,
          "duracion": duracion,
          "categoria": categoria,
          "butacas_libres": butacas_libres,
          "precio": precio
      }

      items.append(item_json)

  return items

def get_instance_selenium():
  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.headless = True
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
  return driver


def scrapear_viajes(driver, ls_origenes, ls_destinos, n_dias):
  """
    Busca todas las combinaciones entre los origenes y el destinos, y scrapena n_dias hacia adelante a partir de mañana

    Args:
      ls_origenes (list): lista con los id de origen
      ls_destinos (list): lista con los id de destino
      n_dias (list): cantidad de dias que se va a escrapear a partir de mañana

    Returns:
      lista de json con los viajes
  """

  getNombreparada = lambda id: df_paradas[df_paradas.id == id].nombre.values[0]
  resultado = []

  for iOrigen in range(len(ls_origenes)):

    id_origen = ls_origenes[iOrigen]
    nombre_origen = getNombreparada(id_origen)

    for iDestino in range(len(ls_destinos)):

      id_destino = ls_destinos[iDestino]
      if id_destino == id_origen: continue
      nombre_destino = getNombreparada(id_destino)
      print(nombre_destino)

      # incializo con la fecha de mañana
      date = datetime.now() + timedelta(1) 
      for j in range(n_dias): 

          year = str(date.year)
          month = ('00' + str(date.month))[-2:]
          day = ('00' + str(date.day))[-2:]
          fecha = f'{year}/{month}/{day}'
          
          result = get_data(driver, id_origen, nombre_origen, id_destino, nombre_destino, fecha)

          resultado.extend(result)
          date += timedelta(days=1)

  return resultado

## scrapeo

In [ ]:
#df_paradas.query('nombre.str.contains("(Entre Rios - ARG)")', engine='python')
ls_origen = [28] # Buenos aires
ls_destino = [176] # Parana
n_dias = 7
driver = get_instance_selenium()
result = scrapear_viajes(driver, ls_origen, ls_destino, n_dias)
driver.quit()
df_results = pd.DataFrame.from_records(result)
df_results.to_excel('viajes.xlsx', encoding = 'utf-8-sig') 

In [ ]:
df_paradas.query('nombre.str.contains("Neuquen")', engine='python')

,id,direccion,latitud,longitud,nombre
26,48,,-40051944200,-7.008050e+10,Piedra del Aguila - Terminal (Neuquen - ARG)
37,63,,-39513156400,-6.929261e+10,Picun Leufu - Terminal (Neuquen - ARG)
47,82,,-38906463200,-7.006786e+10,Zapala - Terminal (Neuquen - ARG)
48,83,,-39948552600,-7.107375e+10,Junin de Los Andes - Terminal (Neuquen - ARG)
49,84,,-38930065800,-6.922579e+10,Cutral Co - Terminal (Neuquen - ARG)
50,85,,-38959141800,-6.810585e+10,Neuquen - Terminal (Neuquen - ARG)
74,122,,-40160464000,-7.135787e+10,San Martin de los Andes - Terminal (Neuquen - ...
120,199,,-40894854000,-7.104029e+10,Villa Llanquin (Neuquen - ARG)
124,209,,-39247285100,-6.879897e+10,El Chocon - Estacion de Servicio (Neuquen - ARG)
129,217,,-40653530600,-7.139987e+10,Villa Traful - Agencia (Neuquen - ARG)


In [ ]:
#df_paradas.query('nombre.str.contains("(Entre Rios - ARG)")', engine='python')
ls_origen = [28] # Buenos aires
ls_argentina = df_paradas.query('nombre.str.contains("Neuquen")', engine='python').id.tolist() # Toda argentina
ls_destino = ls_argentina
n_dias = 7
driver = get_instance_selenium()
result = scrapear_viajes(driver, ls_origen, ls_destino, n_dias)
driver.quit()
df_results = pd.DataFrame.from_records(result)
#df_results.to_excel('Argentina.xlsx', encoding = 'utf-8-sig')

ruta = '/content/drive/MyDrive/viajes/Neuquen_Next_7_days.xlsx'
df_results.to_excel(ruta, index=False, encoding = 'utf-8-sig')


Piedra del Aguila - Terminal (Neuquen - ARG)
Picun Leufu - Terminal (Neuquen - ARG)
Zapala - Terminal (Neuquen - ARG)
Junin de Los Andes - Terminal (Neuquen - ARG)


In [12]:
getNombreparada = lambda id: df_paradas[df_paradas.id == id].nombre.values[0]
driver = get_instance_selenium()

origen_parada = 28
nombre_origen_parada = getNombreparada(origen_parada)
destino_parada = 277
nombre_destino_parada = getNombreparada(destino_parada)
fecha_ida = '2023/03/05'

url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='
driver.get(url)

clean_hora = lambda x: re.search(r'(\d+):(\d+)', str(x)).group() if re.search(r'(\d+):(\d+)', str(x)) else 'None'
clean_precio = lambda x: re.search(r'(\d+)', str(x)).group() if re.search(r'(\d+)', str(x)) else 'None'

items = []
filas  = driver.find_elements(By.CLASS_NAME,"fila_servicio.servicio_fila.idatr")
# recorro los registros de viajes
for fila in filas:
  
  if not "hide " in fila.get_attribute(name="class"):
    id = fila.get_attribute('id')

    empresa_img = fila.find_element(By.TAG_NAME,"td").find_element(By.TAG_NAME, 'img').get_attribute('src')
    salida = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[2]').text.replace('\n', ' ')
    llegada = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[4]').text.replace('\n', ' ')
    escala = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[5]/a').text.replace('\n', ' ')
    duracion = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[6]').text.replace('\n', ' ')
    duracion = clean_hora(duracion)
    categoria = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[7]/ul/li[1]').text
    butacas_libres = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[8]/ul/li[1]').text
    precio = driver.find_element(By.XPATH, f'//*[@id="{id}"]/td[9]/ul/li[1]/span').text
    precio = clean_precio(precio)

    item_json = {
        "origen_parada": origen_parada,
        "nombre_origen_parada": nombre_origen_parada,
        "destino_parada": destino_parada,
        "nombre_destino_parada": nombre_destino_parada,
        "fecha_ida": fecha_ida,
        "empresa_img": empresa_img,
        "salida": salida,
        "llegada": llegada,
        "escala": escala,
        "duracion": duracion,
        "categoria": categoria,
        "butacas_libres": butacas_libres,
        "precio": precio
    }

    items.append(item_json)

driver.quit()
df_results = pd.DataFrame.from_records(items)
df_results



<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="0b776b3f-54a9-4688-9455-e0a9d16451e6")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="a638ecbc-19fb-440c-8ef9-0b9c20af271d")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="b279fbfb-fd91-4a65-a5cf-eedf006c90c0")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="3b234ac4-bba8-4fc2-b5c4-9df2b6654760")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="70591427-8da1-412e-b2e1-f7b3fc606d0c")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="6c6575ac-2aff-4ed5-9159-9cbd5abc8440")>
<selenium.webdriver.remote.webelement.WebElement (session="1233ceb0c0176fae9773435f1f140e85", element="be1eb8cb-6db0-407f-ac28-43cfc9ac1880")>

,origen_parada,nombre_origen_parada,destino_parada,nombre_destino_parada,fecha_ida,empresa_img,salida,llegada,escala,duracion,categoria,butacas_libres,precio
0,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,13:30 Dom 05/03,18:50 Lun 06/03,Transbordo,29:20,Semi Cama,28,22620
1,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,14:05 Dom 05/03,18:50 Lun 06/03,Transbordo,28:45,Cama Ejecutivo,26,25020
2,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,15:00 Dom 05/03,18:50 Lun 06/03,Transbordo,27:50,Cama Ejecutivo,29,25020
3,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,13:30 Dom 05/03,19:50 Lun 06/03,Transbordo,30:20,Semi Cama,28,22620
4,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,14:05 Dom 05/03,19:50 Lun 06/03,Transbordo,29:45,Cama Ejecutivo,26,25020
5,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,13:30 Dom 05/03,20:15 Lun 06/03,Transbordo,30:45,Semi Cama,30,25475
6,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,,,,None,,,None
7,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,,,,None,,,None
8,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,,,,None,,,None
9,28,(BUE) Retiro - Terminal (Capital Federal - ARG...,277,Villa La Angostura - Terminal (Neuquen - ARG),2023/03/05,https://checkout.busplus.com.ar/images/v2/empr...,,,,None,,,None
